import the libraries

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [4]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


clean up the data

In [5]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

In [10]:
covid_data['date'] = pd.to_datetime(covid_data['date'])
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

In [7]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

In [8]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

find the difference in the no. of rows (if any)

In [13]:
rows_1 = covid_data.shape[0]
rows_2 = vaccinations_data.shape[0]
print(rows_1, rows_2, sep=', ')

86785, 42795


In [17]:
covid_data.head(2)

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


Exercise 3.1

In [22]:
max_date = covid_data['date'].max()
min_date = covid_data['date'].min()

print(max_date, min_date, sep=', ')

print(max_date - min_date)

2021-05-29 00:00:00, 2020-01-22 00:00:00
493 days 00:00:00


Exercise 3.2

In [23]:
max_date2 = vaccinations_data['date'].max()
min_date2 = vaccinations_data['date'].min()

print(max_date2, min_date2, sep=', ')

print(max_date2 - min_date2)

2021-09-06 00:00:00, 2020-12-02 00:00:00
278 days 00:00:00


Exercise 3.3

In [25]:
covid_df = covid_data.merge(
    vaccinations_data,
    on = ['country', 'date'],
    how = 'left'
)
covid_df.head()

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86785 entries, 0 to 86784
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 86785 non-null  datetime64[ns]
 1   country                              86785 non-null  object        
 2   confirmed                            86785 non-null  float64       
 3   deaths                               86785 non-null  float64       
 4   recovered                            86785 non-null  float64       
 5   active                               86785 non-null  float64       
 6   daily_confirmed                      86564 non-null  float64       
 7   daily_deaths                         86564 non-null  float64       
 8   daily_recovered                      86564 non-null  float64       
 9   total_vaccinations                   10629 non-null  float64       
 10  people_vac

create a feature death_rate and recover_rate

In [27]:
covid_df['death_rate'] = (covid_df['deaths'] / covid_df['confirmed']) * 100
covid_df['recover_rate'] = (covid_df['recovered'] / covid_df['confirmed']) * 100

In [33]:
covid_df.groupby('country')['death_rate'].agg('describe')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,count,mean,std,min,25%,50%,75%,max
country,,,,,,,,
Afghanistan,461.0,3.436948,1.132104,0.000000,3.185967,3.712734,4.330898,4.424171
Albania,447.0,2.576052,0.950501,0.000000,1.836919,2.351987,2.958842,8.333333
Algeria,460.0,4.485453,3.080389,0.000000,2.686645,3.349556,5.877607,15.784165
Andorra,454.0,2.794071,2.224313,0.000000,1.000233,1.358294,5.358948,6.701708
Angola,436.0,3.934026,3.425928,0.000000,2.325551,2.582838,4.472266,28.571429
...,...,...,...,...,...,...,...,...
West Bank and Gaza,430.0,0.845073,0.240637,0.192678,0.678834,0.866201,1.085527,1.190476
Yemen,415.0,24.647740,6.924957,0.000000,20.442297,28.427740,28.997303,33.333333
Zambia,438.0,1.856188,0.852365,0.000000,1.365368,1.851926,2.269641,5.000000


get all unique values from a column

In [37]:
print('get unique values from specified column:\n', covid_df['country'].unique())

get unique values from specified column:
 ['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Cape Verde' 'Cayman Islands'
 'Central African Republic' 'Chad' 'Channel Islands' 'Chile' 'China'
 'Colombia' 'Comoros' 'Congo' 'Costa Rica' 'Croatia' 'Cuba' 'Curacao'
 'Cyprus' 'Czech Republic' 'Democratic Republic of Congo' 'Denmark'
 'Diamond Princess' 'Djibouti' 'Dominica' 'Dominican Republic'
 'East Timor' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Faroe Islands' 'Fiji'
 'Finland' 'France' 'French Guiana' 'Gabon' 'Gambia' 'Georgia' 'Germany'
 'Ghana' 'Gibraltar' 'Greece' 'Greenland' 'Grenada' 'Guadeloupe' 

Exercise 3.4: max lethal rate for United States

In [40]:
us_only = covid_df[covid_df['country'] == 'United States']
us_only.groupby('country')['death_rate'].agg('max')

country
United States    7.189542
Name: death_rate, dtype: float64

Exercise 3.5: average rate of recovered in Russia?

In [41]:
rus_only = covid_df[covid_df['country'] == 'Russia']
rus_only.groupby('country')['recover_rate'].agg('mean')

country
Russia    67.062735
Name: recover_rate, dtype: float64

export the data set for future use

In [ ]:
covid_df.to_csv('data/combined_covid_data.csv', index = False)